<a href="https://colab.research.google.com/github/AmeerHamza0220/Deep-Learning/blob/main/CNNBasic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.cifar10.load_data()
num_classes = 10
img_rows, img_cols = 32,32
num_channels = 3

In [ ]:
x_train.shape

In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(112,activation='relu',kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01)))


model.add(tf.keras.layers.Dense(num_classes,activation='softmax'))

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-2)

model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
               metrics=[tf.keras.metrics.sparse_categorical_accuracy])
callbacks = [tf.keras.callbacks.TensorBoard('./keras')]
model.fit(x_train, y_train, epochs=2, verbose=1, validation_data=(x_test, y_test), callbacks=callbacks)

Basic stuff using convulution and pooling

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/d/da/Epimachus_meyeri_-Papua_New_Guinea_-male-8.jpg

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
from skimage import io        # Package to simply read images

In [ ]:
from PIL import Image
image = Image.open('/content/Epimachus_meyeri_-Papua_New_Guinea_-male-8.jpg').convert('L').resize((600,600))


In [ ]:
image.shape

In [ ]:
plt.imshow(image, cmap=plt.cm.gray)

In [ ]:
image = tf.keras.preprocessing.image.img_to_array(image)

In [ ]:
image = tf.expand_dims(image, axis=0)
 #tensorflow work on batches 

In [ ]:

print("Tensor shape: {}".format(image.shape))

 we define a 3x3 filter (or kernel) commonly used to blur images (Gaussian blur):

In [ ]:
kernel = tf.constant([[1 / 16, 2 / 16, 1 / 16],
                      [2 / 16, 4 / 16, 2 / 16],
                      [1 / 16, 2 / 16, 1 / 16]], tf.float32, name="gaussian_kernel")

In [ ]:
kernel = tf.expand_dims(tf.expand_dims(kernel, axis=-1), axis=-1)
kernel.shape

In [ ]:
blurred_image = tf.nn.conv2d(image, kernel, strides=[1, 1, 1, 1], padding="SAME")

In [ ]:
blurred_res = blurred_image.numpy()
# We "unbatch" our result by selecting the first (and only) image; we also remove the depth dimension:
blurred_res = blurred_res[0, ..., 0]

plt.imshow(blurred_res, cmap=plt.cm.gray)

a kernel for contour detection is used. This kernel is defined as follows:

In [ ]:
kernel = tf.constant([[-1, -1, -1],
                      [-1,  8, -1],
                      [-1, -1, -1]], tf.float32, name="edge_kernel")
kernel = tf.expand_dims(tf.expand_dims(kernel, axis=-1), axis=-1)

In [ ]:
edge_image = tf.nn.conv2d(image, kernel, strides=[1, 2, 2, 1], padding="SAME")
edge_res = edge_image.numpy()[0, ..., 0]
plt.imshow(edge_res, cmap=plt.cm.gray)

For max-pooling and average-pooling, the values in each window are aggregated into a single output, applying respectively the max or averaging operation. Once again, we use the low-level TensorFlow API to reproduce the results shown in the chapter:

In [ ]:
avg_pooled_image = tf.nn.avg_pool(image, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
avg_res = avg_pooled_image.numpy()[0, ..., 0]
plt.imshow(avg_res, cmap=plt.cm.gray)

In [ ]:
max_pooled_image = tf.nn.max_pool(image, ksize=[1, 10, 10, 1], strides=[1, 2, 2, 1], padding="SAME")
max_res = max_pooled_image.numpy()[0, ..., 0]
plt.imshow(max_res, cmap=plt.cm.gray)

Simple convulutional network

In [ ]:
class LeNet(tf.keras.Model):
    def __init__(self,num_classes):
        super(LeNet, self).__init__()
        self.conv1=tf.keras.layers.Conv2D(20,kernel_size=5,padding="same",activation='relu')
        self.conv2=tf.keras.layers.Conv2D(30,kernel_size=5,activation='relu')
        self.maxpool=tf.keras.layers.MaxPooling2D(pool_size=2)
        self.flatten=tf.keras.layers.Flatten()
        self.dense1=tf.keras.layers.Dense(400,activation='relu')
        self.dense2=tf.keras.layers.Dense(num_classes,activation='relu')
    def call(self,input):
        x=self.conv1(input)
        x=self.maxpool(x)
        x=self.conv2(x)
        x=self.maxpool(x)
        x=self.flatten(x)
        x=self.dense1(x)
        x=self.dense2(x)
        return x

In [ ]:
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.cifar10.load_data()
num_classes = 10
img_rows, img_cols = 32,32
num_channels = 3

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
class LeNet5(Model):
    
    def __init__(self, num_classes):
        """
        Initialize the model.
        :param num_classes:     Number of classes to predict from
        """
        super(LeNet5, self).__init__()
        # We instantiate the various layers composing LeNet-5:
        # self.conv1 = SimpleConvolutionLayer(6, kernel_size=(5, 5))
        # self.conv2 = SimpleConvolutionLayer(16, kernel_size=(5, 5))
        # ... or using the existing and (recommended) Conv2D class:
        self.conv1 = Conv2D(6, 5, padding='same', activation='relu')
        self.conv2 = Conv2D(16, 5, activation='relu')
        self.max_pool = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='relu')
        self.dense2 = Dense(84, activation='relu')
        self.dense3 = Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        """
        Call the layers and perform their operations on the input tensors
        :param inputs:  Input tensor
        :return:        Output tensor
        """
        x = self.max_pool(self.conv1(inputs))        # 1st block
        x = self.max_pool(self.conv2(x))             # 2nd block
        x = self.flatten(x)
        x = self.dense3(self.dense2(self.dense1(x))) # dense layers
        return x

In [ ]:
input_shape=32,32,3


model = LeNet5(10)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.sparse_categorical_accuracy])
callbacks = [
    # Callback to interrupt the training if the validation loss (`val_loss`) stops improving for over 3 epochs:
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    # Callback to log the graph, losses and metrics into TensorBoard (saving log files in `./logs` directory):
    tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True)]
# batched_input_shape = tf.TensorShape((None, *input_shape))
# model.build(input_shape=batched_input_shape)
# model.summary()

In [ ]:
x_train=tf.cast(x_train, tf.float32)
x_test=tf.cast(x_test, tf.float32)

history = model.fit(x_train, y_train,
                    batch_size=32, epochs=10, validation_data=(x_test, y_test), 
                    verbose=1,  # change to `verbose=1` to get a progress bar
                                # (we opt for `verbose=2` here to reduce the log size)
                    callbacks=callbacks)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

Improving above lenet with batchnorm and l1 regularization

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
class LeNet(Model):
    
    def __init__(self, num_classes):
        """
        Initialize the model.
        :param num_classes:     Number of classes to predict from
        """
        super(LeNet, self).__init__()
        # We instantiate the various layers composing LeNet-5:
        # self.conv1 = SimpleConvolutionLayer(6, kernel_size=(5, 5))
        # self.conv2 = SimpleConvolutionLayer(16, kernel_size=(5, 5))
        # ... or using the existing and (recommended) Conv2D class:
        self.conv1 = Conv2D(6, 5, padding='same', activation='relu',   kernel_regularizer=tf.keras.regularizers.l1(0.01),)
        self.bn1=tf.keras.layers.BatchNormalization()
        self.conv2 = Conv2D(16, 5, activation='relu',   kernel_regularizer=tf.keras.regularizers.l1(0.01),)
        self.bn2=tf.keras.layers.BatchNormalization()

        self.max_pool = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='relu',   kernel_regularizer=tf.keras.regularizers.l1(0.01),)
        self.dense2 = Dense(84, activation='relu',   kernel_regularizer=tf.keras.regularizers.l1(0.01),)
        self.dense3 = Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        """
        Call the layers and perform their operations on the input tensors
        :param inputs:  Input tensor
        :return:        Output tensor
        """
        x = self.max_pool(self.conv1(inputs))        # 1st block
        x = self.max_pool(self.conv2(x))             # 2nd block
        x = self.flatten(x)
        x = self.dense3(self.dense2(self.dense1(x))) # dense layers
        return x

In [ ]:
input_shape=32,32,3


model = LeNet(10)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.sparse_categorical_accuracy])
callbacks = [
    # Callback to interrupt the training if the validation loss (`val_loss`) stops improving for over 3 epochs:
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    # Callback to log the graph, losses and metrics into TensorBoard (saving log files in `./logs` directory):
    tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True)]
# batched_input_shape = tf.TensorShape((None, *input_shape))
# model.build(input_shape=batched_input_shape)
# model.summary()

In [ ]:
x_train=tf.cast(x_train, tf.float32)
x_test=tf.cast(x_test, tf.float32)

history = model.fit(x_train, y_train,
                    batch_size=32, epochs=10, validation_data=(x_test, y_test), 
                    verbose=1,  # change to `verbose=1` to get a progress bar
                                # (we opt for `verbose=2` here to reduce the log size)
                    callbacks=callbacks)



Now we will implement Resnet

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Activation, Dense, Flatten, Conv2D, MaxPooling2D, 
    GlobalAveragePooling2D, AveragePooling2D, BatchNormalization, add)
import tensorflow.keras.regularizers as regulizers
import functools

class ConvWithBatchNorm(tf.keras.layers.Conv2D):
    """ Convolutional layer with batch normalization"""

    def __init__(self, activation='relu', name='convbn', **kwargs):
        """
        Initialize the layer. 
        :param activation:   Activation function (name or callable)
        :param name:         Name suffix for the sub-layers.
        :param kwargs:       Mandatory and optional parameters of tf.keras.layers.Conv2D
        """
        
        self.activation = Activation(
            activation, name=name + '_act') if activation is not None else None
        
        super().__init__(activation=None, name=name + '_c', **kwargs)
        
        self.batch_norm = BatchNormalization(axis=-1, name=name + '_bn')

    def call(self, inputs, training=None):
        """
        Call the layer. 
        :param inputs:         Input tensor to process
        :param training:       Flag to let TF knows if it is a training iteration or not
                               (this will affect the behavior of BatchNorm)
        :return:               Convolved tensor
        """
        x = super().call(inputs)
        x = self.batch_norm(x, training=training)
        if self.activation is not None:
            x = self.activation(x)
        return x


class ResidualMerge(tf.keras.layers.Layer):
    """ Layer to merge the original tensor and the residual one in residual blocks"""

    def __init__(self, name='block', **kwargs):
        """
        Initialize the layer. 
        :param activation:   Activation function (name or callable)
        :param name:         Name suffix for the sub-layers.
        :param kwargs:       Optional parameters of tf.keras.layers.Conv2D
        """
        
        super().__init__(name=name)
        self.shortcut = None
        self.kwargs = kwargs
        
    def build(self, input_shape):
        x_shape = input_shape[0]
        x_residual_shape = input_shape[1]
        if x_shape[1] == x_residual_shape[1] and \
           x_shape[2] == x_residual_shape[2] and \
           x_shape[3] == x_residual_shape[3]:
            self.shortcut = functools.partial(tf.identity, name=self.name + '_shortcut')
        else:
            strides = (
                int(round(x_shape[1] / x_residual_shape[1])), # vertical stride
                int(round(x_shape[2] / x_residual_shape[2]))  # horizontal stride
            )
            x_residual_channels = x_residual_shape[3]
            self.shortcut = ConvWithBatchNorm(
                filters=x_residual_channels, kernel_size=(1, 1), strides=strides,
                activation=None, name=self.name + '_shortcut_c', **self.kwargs)
        

    def call(self, inputs):
        """
        Call the layer. 
        :param inputs:         Tuple of two input tensors to merge
        :return:               Merged tensor
        """
        x, x_residual = inputs
        
        x_shortcut = self.shortcut(x)
        x_merge = add([x_shortcut, x_residual])
        return x_merge

class BasicResidualBlock(tf.keras.Model):
    """ Basic residual block"""

    def __init__(self, filters=16, kernel_size=1, strides=1, activation='relu',
                 kernel_initializer='he_normal', kernel_regularizer=regulizers.l2(1e-4),
                 name='res_basic', **kwargs):
        """
        Initialize the layer. 
        :param filters:                 Number of filters
        :param kernel_size:             Kernel size
        :param strides:                 Convolution strides
        :param activation:              Activation function (name or callable)
        :param kernel_initializer:      Kernel initialisation method name
        :param kernel_regularizer:      Kernel regularizer
        :param name:                    Name suffix for the sub-layers.
        :param kwargs:                  Optional parameters of tf.keras.layers.Conv2D
        """
        super().__init__(name=name)
        
        self.conv_1 = ConvWithBatchNorm(
            filters=filters, kernel_size=kernel_size, activation=activation, padding='same',
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer,
            strides=strides, name=name + '_cb_1', **kwargs)
        
        self.conv_2 = ConvWithBatchNorm(
            filters=filters, kernel_size=kernel_size, activation=None, padding='same',
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer,
            strides=1, name=name + '_cb_2', **kwargs)
        
        self.merge = ResidualMerge(
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, 
            name=name)
        
        self.activation = Activation(activation, name=name + '_act')

    def call(self, inputs, training=None):
        """
        Call the block. 
        :param inputs:         Input tensor to process
        :param training:       Flag to let TF knows if it is a training iteration or not
                               (this will affect the behavior of BatchNorm)
        :return:               Block output tensor
        """
        x = inputs
        # Residual path:
        x_residual = self.conv_1(x, training=training)
        x_residual = self.conv_2(x_residual, training=training)
        
        # Merge residual result with original tensor:
        x_merge = self.merge([x, x_residual])
        x_merge = self.activation(x_merge)
        return x_merge

class ResidualBlockWithBottleneck(tf.keras.Model):
    """ Residual block with bottleneck, recommended for deep ResNets (depth > 34)"""
    def __init__(self, filters=16, kernel_size=1, strides=1, activation='relu',
                 kernel_initializer='he_normal', kernel_regularizer=regulizers.l2(1e-4),
                 name='res_basic', **kwargs):
        """
        Initialize the block. 
        :param filters:                 Number of filters
        :param kernel_size:             Kernel size
        :param strides:                 Convolution strides
        :param activation:              Activation function (name or callable)
        :param kernel_initializer:      Kernel initialisation method name
        :param kernel_regularizer:      Kernel regularizer
        :param name:                    Name suffix for the sub-layers.
        :param kwargs:                  Optional parameters of tf.keras.layers.Conv2D
        """
        super().__init__(name=name)
        
        self.conv_0 = ConvWithBatchNorm(
            filters=filters, kernel_size=1, activation=activation, padding='valid',
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer,
            strides=1, name=name + '_cb_0', **kwargs)
        
        self.conv_1 = ConvWithBatchNorm(
            filters=filters, kernel_size=kernel_size, activation=activation, padding='same',
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer,
            strides=strides, name=name + '_cb_1', **kwargs)
        
        self.conv_2 = ConvWithBatchNorm(
            filters=4 * filters, kernel_size=1, activation=None, padding='valid',
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer,
            strides=1, name=name + '_cb_2', **kwargs)
        
        self.merge = ResidualMerge(
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, 
            name=name)
        
        self.activation = Activation(activation, name=name + '_act')

    def call(self, inputs, training=None):
        """
        Call the layer. 
        :param inputs:         Input tensor to process
        :param training:       Flag to let TF knows if it is a training iteration or not
                               (this will affect the behavior of BatchNorm)
        :return:               Block output tensor
        """
        x = inputs
        # Residual path:
        x_residual = self.conv_0(x, training=training)
        x_residual = self.conv_1(x_residual, training=training)
        x_residual = self.conv_2(x_residual, training=training)
        
        # Merge residual result with original tensor:
        x_merge = self.merge([x, x_residual])
        x_merge = self.activation(x_merge)
        return x_merge

class ResidualMacroBlock(tf.keras.models.Sequential):
    """ Macro-block, chaining multiple residual blocks (as a Sequential model)"""

    def __init__(self, block_class=ResidualBlockWithBottleneck, repetitions=3, 
                 filters=16, kernel_size=1, strides=1, activation='relu',
                 kernel_initializer='he_normal', kernel_regularizer=regulizers.l2(1e-4),
                 name='res_macroblock', **kwargs):
        """
        Initialize the block. 
        :param block_class:             Block class to be used.
        :param repetitions:             Number of times the block should be repeated inside.
        :param filters:                 Number of filters
        :param kernel_size:             Kernel size
        :param strides:                 Convolution strides
        :param activation:              Activation function (name or callable)
        :param kernel_initializer:      Kernel initialisation method name
        :param kernel_regularizer:      Kernel regularizer
        :param name:                    Name suffix for the sub-layers.
        :param kwargs:                  Optional parameters of tf.keras.layers.Conv2D
        """
        super().__init__(
            [block_class(
                 filters=filters, kernel_size=kernel_size, activation=activation,
                 strides=strides if i == 0 else 1, name="{}_{}".format(name, i),
                 kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)
             for i in range(repetitions)], 
            name=name)
        
class ResNet(tf.keras.models.Sequential):
    """ ResNet model for classification"""

    def __init__(self, input_shape, num_classes=1000, 
                 block_class=ResidualBlockWithBottleneck, repetitions=(2, 2, 2, 2),
                 kernel_initializer='he_normal', kernel_regularizer=regulizers.l2(1e-4),
                 name='resnet'):
        """
        Initialize a ResNet model for classification.
        :param input_shape:             Input shape (e.g. (224, 224, 3))
        :param num_classes:             Number of classes to predict
        :param block_class:             Block class to be used
        :param repetitions:             List of repetitions for each macro-blocks the network should contain
        :param kernel_initializer:      Kernel initialisation method name
        :param kernel_regularizer:      Kernel regularizer
        :param name:                    Model's name
        :return:                        ResNet model.
        """
    
        filters = 64
        strides = 2
    
        super().__init__(
            # Initial conv + max-pool layers:
            [Input(shape=input_shape, name='input'),
             ConvWithBatchNorm(
                 filters=filters, kernel_size=7, activation='relu', padding='same', strides=strides,
                 kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer,
                 name='conv'),
             MaxPooling2D(pool_size=3, strides=strides, padding='same', name='max_pool')
            ] + \
            # Residual blocks:
            [ResidualMacroBlock(
                 block_class=block_class, repetitions=repet, 
                 filters=min(filters * (2 ** i), 1024), kernel_size=3, activation='relu',
                 strides=strides if i != 0 else 1, name='block_{}'.format(i),
                 kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)
             for i, repet in enumerate(repetitions)
            ] + \
            # Final layers leading to classification output:
            [GlobalAveragePooling2D(name='avg_pool'),
             Dense(units=num_classes, kernel_initializer=kernel_initializer, activation='softmax')
            ], name=name)


In [ ]:
class ResNet18(ResNet):
    def __init__(self, input_shape, num_classes=1000, name='resnet18',
                 kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4)):
        super().__init__(input_shape, num_classes, 
                         block_class=BasicResidualBlock, repetitions=(2, 2, 2, 2),
                         kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)
        
class ResNet34(ResNet):
    def __init__(self, input_shape, num_classes=1000, name='resnet34',
                 kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4)):
        super().__init__(input_shape, num_classes, 
                         block_class=BasicResidualBlock, repetitions=(3, 4, 6, 3),
                         kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)
        
class ResNet50(ResNet):
    def __init__(self, input_shape, num_classes=1000, name='resnet50',
                 kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4)):
        super().__init__(input_shape, num_classes, 
                         block_class=ResidualBlockWithBottleneck, repetitions=(3, 4, 6, 3),
                         kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)

class ResNet101(ResNet):
    def __init__(self, input_shape, num_classes=1000, name='resnet101',
                 kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4)):
        super().__init__(input_shape, num_classes, 
                         block_class=ResidualBlockWithBottleneck, repetitions=(3, 4, 23, 3),
                         kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)

class ResNet152(ResNet):
    def __init__(self, input_shape, num_classes=1000, name='resnet152',
                 kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4)):
        super().__init__(input_shape, num_classes, 
                         block_class=ResidualBlockWithBottleneck, repetitions=(3, 8, 36, 3),
                         kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)

In [ ]:
input_shape = [32, 32, 3] 
num_classes=10
model = ResNet50(input_shape, num_classes)
model.summary(line_length=80, positions=[.5, .85, 1.])

In [ ]:
optimizer = tf.keras.optimizers.Adam() #tf.keras.optimizers.SGD(momentum=0.9, nesterov=True)

accuracy_metric = tf.metrics.SparseCategoricalAccuracy(name='acc')
top5_accuracy_metric = tf.metrics.SparseTopKCategoricalAccuracy(k=5, name='top5_acc')
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', 
                 metrics=[accuracy_metric, top5_accuracy_metric])

In [ ]:
history = model.fit(x_train, y_train,  
                       epochs=10,
                        validation_data=(x_test, y_test),
                       verbose=1)

Transfor learning vision tranformer

In [ ]:
! pip install -q kaggle


In [ ]:
from google.colab import files

files.upload()

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!pip install --upgrade --force-reinstall --no-deps kaggle

!kaggle datasets download -d alxmamaev/flowers-recognition


In [ ]:
!unzip -q flowers-recognition.zip
!rm flowers-recognition.zip

In [ ]:
batch_size = 32
img_height = 224
img_width = 224
num_classes = 5

train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    "/content/flowers", labels='inferred',shuffle=True,image_size=(img_height,
    img_width),subset="training",
 validation_split=0.2,seed=123,
 batch_size=32

)
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
    "/content/flowers", labels='inferred',shuffle=True,image_size=(img_height,
    img_width),subset="validation",
 validation_split=0.2,seed=123,
 batch_size=32

)

In [ ]:
import tensorflow_hub as hub
url="https://tfhub.dev/sayakpaul/vit_b8_fe/1"
hub_feature_extractor = hub.KerasLayer(
    url, 
    trainable=False,                              # Flag to set the layers as trainable or not
    input_shape=(224,224,3),   # Expected input shape.
    output_shape=1000, # Output shape [batch_size, 2048].
    dtype=tf.float32)    

In [ ]:
model=tf.keras.Sequential(
    [hub_feature_extractor,
     tf.keras.layers.Dense(num_classes,activation='softmax')
     
    ]
)
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(name='acc'),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top5_acc')
    ])

In [ ]:
model.fit(
    train_ds,  epochs=10,
    validation_data=val_ds,
    verbose=1)

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

def load_image(image_path, size):
    """
    Load an image as a Numpy array.
    :param image_path:  Path of the image
    :param size:        Target size
    :return             Image array, normalized between 0 and 1
    """
    image = img_to_array(load_img(image_path, target_size=size)) / 255.
    return image


def process_predictions(class_probabilities, class_readable_labels, k=5):
    """
    Process a batch of predictions from our estimator.
    :param class_probabilities:     Prediction results returned by the Keras classifier for a batch of data
    :param class_readable_labels:   List of readable-class labels, for display
    :param k:                       Number of top predictions to consider
    :return                         Readable labels and probabilities for the predicted classes
    """
    topk_labels, topk_probabilities = [], []
    for i in range(len(class_probabilities)):
        # Getting the top-k predictions:
        topk_classes = sorted(np.argpartition(class_probabilities[i], -k)[-k:])
    
        # Getting the corresponding labels and probabilities:
        topk_labels.append([class_readable_labels[predicted] for predicted in topk_classes])
        topk_probabilities.append(class_probabilities[i][topk_classes])
    
    return topk_labels, topk_probabilities


def display_predictions(images, topk_labels, topk_probabilities):
    """
    Plot a batch of predictions.
    :param images:                  Batch of input images
    :param topk_labels:             String labels of predicted classes
    :param topk_probabilities:      Probabilities for each class
    """
    num_images = len(images)
    num_images_sqrt = np.sqrt(num_images)
    plot_cols = plot_rows = int(np.ceil(num_images_sqrt))
    
    figure = plt.figure(figsize=(13,10))
    grid_spec = gridspec.GridSpec(plot_cols, plot_rows)
    
    for i in range(num_images):
        img, pred_labels, pred_proba = images[i], topk_labels[i], topk_probabilities[i]
        # Shortening the labels to better fit in the plot:
        pred_labels = [label.split(',')[0][:20] for label in pred_labels]
        
        grid_spec_i = gridspec.GridSpecFromSubplotSpec(3, 1, subplot_spec=grid_spec[i], 
                                                       hspace=0.1)
        
        # Drawing the input image:
        ax_img = figure.add_subplot(grid_spec_i[:2])
        ax_img.axis('off')
        ax_img.imshow(img)
        ax_img.autoscale(tight=True)
        
        # Plotting a bar chart for the predictions:
        ax_pred = figure.add_subplot(grid_spec_i[2])
        ax_pred.spines['top'].set_visible(False)
        ax_pred.spines['right'].set_visible(False)
        ax_pred.spines['bottom'].set_visible(False)
        ax_pred.spines['left'].set_visible(False)
        y_pos = np.arange(len(pred_labels))
        ax_pred.barh(y_pos, pred_proba, align='center')
        ax_pred.set_yticks(y_pos)
        ax_pred.set_yticklabels(pred_labels)
        ax_pred.invert_yaxis()
        
    plt.tight_layout()
    plt.show()

In [ ]:
model.predict_on_batch(val_ds[1])